In [4]:
!pip install bert_score

  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cufft_cu12-1

In [ ]:
import pandas as pd
import warnings
from bert_score import score
import logging
import sys
import kagglehub


kagglehub.dataset_download('adityaav80/evaulation')
start = 15001
end = 20000

# Suppress warnings and logs
warnings.filterwarnings("ignore")
logging.getLogger("transformers").setLevel(logging.ERROR)

# Load CSV
df = pd.read_csv("/kaggle/input/evaulation/combined_blip2_evaluation.csv")

print(len(df))

# Slice the DataFrame
df_slice = df.iloc[start:end].copy()

# Add semantic metric columns
df_slice["bertscore_precision_short"] = 0.0
df_slice["bertscore_recall_short"] = 0.0
df_slice["bertscore_f1_short"] = 0.0

df_slice["bertscore_precision_long"] = 0.0
df_slice["bertscore_recall_long"] = 0.0
df_slice["bertscore_f1_long"] = 0.0

# Compute BERTScore for the slice
for idx, row in df_slice.iterrows():
    short_pred = str(row["blip2_prediction_short"])
    long_pred = str(row["blip2_prediction_long"])
    answer = str(row["answer"])

    P_s, R_s, F1_s = score([short_pred], [answer], lang='en', verbose=False)
    df_slice.at[idx, "bertscore_precision_short"] = P_s.item()
    df_slice.at[idx, "bertscore_recall_short"] = R_s.item()
    df_slice.at[idx, "bertscore_f1_short"] = F1_s.item()

    P_l, R_l, F1_l = score([long_pred], [answer], lang='en', verbose=False)
    df_slice.at[idx, "bertscore_precision_long"] = P_l.item()
    df_slice.at[idx, "bertscore_recall_long"] = R_l.item()
    df_slice.at[idx, "bertscore_f1_long"] = F1_l.item()

    if idx % 50 == 0:
        print(f"{idx} rows complete")

# Save part
df_slice.to_csv(f"blip2_semantic_part_{start}_{end}.csv", index=False)
print(f"Saved part {start} to {end}")

50040


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-05-14 04:23:02.034182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747196582.268610      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747196582.339380      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

15050 rows complete
15100 rows complete
